# Overview of Results

Do some analysis on results, assume `kikai` / `not_yeti` have done the hardwork and properly normalised names etc.

## Using Notebook

Ensure prerequisites:

```shell
pip install -r src/visualisations/extra-requirements.txt
```

In [1]:
# Update results etc, remember to update tournaments.yml for what you care about
import os
import sys
from pathlib import Path
import subprocess


env = dict(os.environ)
env["PYTHONUNBUFFERED"] = "1"

proc = subprocess.Popen(
    [sys.executable, "-u", Path("src", "netrunner_results.py")],  # Use same kernel
    cwd=Path("..", ".."),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

try:
    for line in proc.stdout:
        print(line, end="")
finally:
    if proc.stdout:
        proc.stdout.close()
    rc = proc.wait()

[2025-08-02] EMEA Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4110]
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
[2025-08-09] APAC Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4118]
nrdb_id not found for Dia [3]
[2025-08-16] APAC Online Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4134]
nrdb_id not found for Lia [5]
[2025-08-23] America Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4129]
nrdb_id not found for sam keilback [14]
nrdb_id not found for Steve [9]
[2025-08-30] EMEA Online Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4149]
could not find identity: None
could not find identity: None


In [2]:
# Data load and utilities
import yaml
import pandas as pd
import numpy as np

faction_colors = {
    "Anarch": "#e21b3c",
    "Criminal": "#0011ff",
    "Shaper": "#2ca02c",
    "HB": "#9467bd",
    "Jinteki": "#e21b3c",
    "Weyland": "#2ca02c",
    "NBN": "#fbff0e",
    "Mini-faction": "#7f7f7f",
    "Neutral Corp": "#7f7f7f",
    "Neutral Runner": "#7f7f7f",
}

LOW_GAME_CUTOFF_P = 0.035

BASE_DIR = Path("..", "..")
DATA_DIR = Path(BASE_DIR, "OUTPUT")
RESULTS_DIR = Path(DATA_DIR, "results")
STANDINGS_DIR = Path(DATA_DIR, "standings")


with open(Path(BASE_DIR, "src", "netrunner", "identities.yml"), "r", encoding="utf-8") as f:
    identities = yaml.safe_load(f)

id_to_faction = {}
for name, data in identities.items():
    faction = data.get("faction")
    id_to_faction[name] = faction
    for alt in data.get("alt_names", []) or []:
        id_to_faction[alt] = faction

valid_identities = set(id_to_faction.keys())


res_frames = []
FILES = sorted(RESULTS_DIR.rglob("*.csv"))
for fp in FILES:
    df = pd.read_csv(
        fp,
        header=0,
        parse_dates=["date"],
        # keep_default_na=False,
        encoding="utf-8",
        engine="python",
    )

    res_frames.append(df)

results = pd.concat(res_frames, ignore_index=True)

# Label byes
mask_invalid = ~results["corp_id"].isin(valid_identities) | ~results["runner_id"].isin(valid_identities)
results.loc[mask_invalid, "result"] = "bye"

# Remove all byes and IDs
mask_byes_ids = results["result"].isin(["bye", "ID"])
results = results[~mask_byes_ids]

standing_frames = []
FILES = sorted(STANDINGS_DIR.rglob("*.csv"))
for fp in FILES:
    df = pd.read_csv(
        fp,
        header=0,
        parse_dates=["date"],
        # keep_default_na=False,
        encoding="utf-8",
        engine="python",
    )

    standing_frames.append(df)

standings = pd.concat(standing_frames, ignore_index=True)

tournaments = results["tournament"].dropna().unique()

print(f"Number of tournaments: {len(FILES)}")
for t in sorted(tournaments):
    print(" -", t)
print(f"Number of games: {len(results)}")
print(f"  Removed {sum(mask_byes_ids)} byes and IDs")

Number of tournaments: 5
 - APAC Continentals 2025
 - APAC Online Continentals 2025
 - America Continentals 2025
 - EMEA Continentals 2025
 - EMEA Online Continentals 2025
Number of games: 1846
  Removed 35 byes and IDs


## Follow Pro Players

Cut data to follow only the best players for some definition of "best"

In [3]:
import pandas as pd
import re

split_type = "any-swiss-top20"

cutters = standings.copy()
cutter_results = results.copy()

# Who made it?
cutters["top_cut_rank_num"] = pd.to_numeric(cutters["top_cut_rank"], errors="coerce")
cutters = cutters[cutters["top_cut_rank_num"].notna()]

# (tournament_id, player) pairs of people who made a cut and what tournament it was in
cut_player_tournament = cutters[["tournament_id", "name"]].drop_duplicates().rename(columns={"name": "player"})

if split_type == "tournament-cutter":
    # Defines a cutter as someone who made top cut in the *specific tournament*
    cut_idx = pd.MultiIndex.from_frame(cut_player_tournament)

    corp_match = results.set_index(["tournament_id", "corp_player"]).index.isin(cut_idx)
    runner_match = results.set_index(["tournament_id", "runner_player"]).index.isin(cut_idx)
    mask = corp_match | runner_match

    cutter_results = results.loc[mask].copy()

    # Tag
    corp_idx = pd.MultiIndex.from_arrays([cutter_results["tournament_id"], cutter_results["corp_player"]])
    runner_idx = pd.MultiIndex.from_arrays([cutter_results["tournament_id"], cutter_results["runner_player"]])
    cutter_results["corp_is_cutter"] = corp_idx.isin(cut_idx)
    cutter_results["runner_is_cutter"] = runner_idx.isin(cut_idx)

elif split_type == "any-cutter":
    # Defines a cutter as someone who made the top cut in *any* tournament (include their games from other tournaments)
    cut_idx = cut_player_tournament["player"]

    corp_match = results["corp_player"].isin(cut_idx)
    runner_match = results["runner_player"].isin(cut_idx)
    mask = corp_match | runner_match

    cutter_results = results.loc[mask].copy()

    # Tag
    cutter_results["corp_is_cutter"] = cutter_results["corp_player"].isin(cut_idx)
    cutter_results["runner_is_cutter"] = cutter_results["runner_player"].isin(cut_idx)

elif split_type.startswith("any-swiss-top"):
    m = re.match(r"any-swiss-top(\d+)", split_type)
    if not m:
        raise ValueError(f"Could not parse N from split_type='{split_type}'")
    SWISS_TOP_N = int(m.group(1))

    swiss = standings.copy()
    swiss["swiss_rank_num"] = pd.to_numeric(swiss["swiss_rank"], errors="coerce")
    swiss = swiss[swiss["swiss_rank_num"].notna() & (swiss["swiss_rank_num"] <= SWISS_TOP_N)]

    swiss_top_players = swiss[["name"]].drop_duplicates()
    top_idx = swiss_top_players["name"]

    corp_match = results["corp_player"].isin(top_idx)
    runner_match = results["runner_player"].isin(top_idx)
    mask = corp_match | runner_match

    cutter_results = results.loc[mask].copy()

    cutter_results["corp_is_cutter"] = cutter_results["corp_player"].isin(top_idx)
    cutter_results["runner_is_cutter"] = cutter_results["runner_player"].isin(top_idx)

else:
    raise ValueError("Didn't understand split type")

low_game_cutoff_nb = np.ceil(LOW_GAME_CUTOFF_P * len(cutter_results))

if split_type.startswith("any-swiss-top"):
    unique_players = swiss_top_players["name"].nunique()
    names_preview = sorted(swiss_top_players["name"].unique().tolist())
else:
    unique_players = cut_player_tournament["player"].nunique()
    names_preview = sorted(cut_player_tournament["player"].unique().tolist())

print(f"Cutters (unique): {unique_players}")
print(f"Games kept: {len(cutter_results)}")
print(names_preview)
print(f"Low game cutoff: {low_game_cutoff_nb} games")

Cutters (unique): 90
Games kept: 864
['AlPi', 'AugustusCaesar', 'Bemi', 'Bridgeman', 'BucketHatBen', 'Chicken Slayer', 'ChiptheRipper', 'ChonkySeal', 'CyberspacePanda', 'DeeR', 'Dia', 'ExperimentalDataCore', 'F3nr1s', 'Frederica Bernkastel', 'Fridan', 'Gargareth', 'Ghost Meat', 'Gloompunk', 'Icecreamcollege', 'Jai', 'Joey', 'KennyG', 'Kieran', 'Kysra', 'Leon', 'Lia', 'MattOhNo', 'Matuszczak', 'Miles', 'Minstrel', 'MotionBlur', 'MrVellis', 'NecroB', 'NewPuldrix', 'NotAgain', 'Ollie', 'Pinsel', 'Porkobolo, the Roaring Pig', 'Redino987', 'Rjorb', 'RotomAppliance', 'Sauc3', 'SnarkySaxophonist', 'Sokka', 'Steve', 'Stwyde', 'Styx', 'Sumpurnis', 'TorpedoTyrus', 'Vale', 'Wenjong', 'Ying64', 'ZomZraft', 'analyzechris', 'ant1', 'bing005', 'bluestar', 'cablooshe', 'chaosjuggler', 'chaosjuggler (Jess)', 'coldlava', 'cros', 'davz131', 'eBentl', 'emaaaly', 'hams', 'laura_42', 'lunari', 'mallory (l0velace)', 'maninthemoon', 'metrixgo', 'nbkelly (oracle for the king)', 'nervousnightjar', 'not_yeti', '

In [4]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, Div
from bokeh.layouts import column
from bokeh.io import output_notebook, output_file, save

output_notebook()


def _apply_low_game_flags(df: pd.DataFrame, cutoff: int) -> pd.DataFrame:
    d = df.copy()
    d["is_low"] = d["games"] < cutoff
    d["alpha"] = np.where(d["is_low"], 0.35, 1.0)  # For fading on graph
    d["winrate_pct"] = (d["winrate"] * 100).round(2)

    d["low_label"] = np.where(d["is_low"], "<b>(LOW)</b>", "")  # label for tooltip
    return d


def _abbrev_list(items, max_items=7, max_chars=400):
    """Join unique with <br>, trim length, and add '+N more…' if overflowing"""
    uniq = list(dict.fromkeys(items))
    shown = uniq[:max_items]
    s = "<br>".join(shown)
    truncated = False
    if len(s) > max_chars:
        s = s[: max_chars - 1].rstrip() + ".."
        truncated = True
    more = max(0, len(uniq) - len(shown))
    if more or truncated:
        s += f"<br><em>+{more} more…</em>" if more else "<br><em>…</em>"
    return s


def _agg_identity_winrate(df: pd.DataFrame, who: str) -> pd.DataFrame:
    d = df.copy()
    d["result"] = d["result"].astype(str).str.lower()

    id_col = f"{who}_id"
    faction_col = f"{who}_faction"
    flag_col = f"{who}_is_cutter"
    player_col = (
        f"{who}_player" if f"{who}_player" in d.columns else ("corp_player" if who == "corp" else "runner_player")
    )

    # Keep only rows where the relevant side was playing
    d = d[d[flag_col]]

    # Add sample label
    d["sample_pair"] = d[player_col].astype(str).str.strip() + " - " + d["tournament"].astype(str)

    grp = d.groupby([id_col, faction_col], dropna=False, as_index=False).agg(
        games=("result", "size"),
        wins=("result", lambda s: (s == who).sum()),
        pairs=("sample_pair", lambda s: _abbrev_list(s)),
    )

    grp["winrate"] = grp["wins"] / grp["games"]
    grp.rename(columns={id_col: "identity", faction_col: "faction", "pairs": "examples"}, inplace=True)
    return grp.sort_values(["winrate", "games"], ascending=[False, False])


def _top_n(df: pd.DataFrame, n: int = 30) -> pd.DataFrame:
    if df.empty:
        return df

    top_by_games = df.sort_values(["games", "winrate"], ascending=[False, False]).head(n).copy()
    top_by_games.sort_values(["winrate", "games", "identity"], ascending=[False, False, True], inplace=True)
    return top_by_games


def _hbar_chart(df: pd.DataFrame, title: str):
    d = df.copy().reset_index(drop=True)
    d = _apply_low_game_flags(df, low_game_cutoff_nb)
    d["identity"] = d["identity"].fillna("(unknown)").astype(str)
    d["winrate_pct"] = (d["winrate"] * 100).round(2)
    d["color"] = d["faction"].map(faction_colors).fillna("#cccccc")

    categories = d["identity"].tolist()
    src = ColumnDataSource(d)

    p = figure(
        height=max(320, 22 * len(categories)),
        sizing_mode="stretch_width",
        y_range=categories,
        x_range=(0, max(1.0, float(d["winrate"].max()) * 1.05)),
        title=title,
        toolbar_location="above",
    )

    bars = p.hbar(y="identity", right="winrate", height=0.7, source=src, color="color", alpha="alpha")

    # p.add_tools(
    #     HoverTool(
    #         renderers=[bars],
    #         tooltips=[
    #             ("Identity", "@identity"),
    #             ("Faction", "@faction"),
    #             ("Win rate", "@winrate_pct%"),
    #             ("Wins", "@wins"),
    #             ("Games", "@games"),
    #             ("Examples", "@examples"),
    #         ],
    #     )
    # )

    p.add_tools(
        HoverTool(
            renderers=[bars],
            tooltips="""
            <div style="font-size: 12px; line-height: 1.25;">
            <div><b>@identity</b></div>
            <div><b>Faction:</b> @faction</div>
            <div><b>Win rate:</b> @winrate_pct%</div>
            <div><b>Wins/Games:</b> @wins / @games @{low_label}{safe}</div>
            <div style="margin-top:4px;"><b>Examples:</b><br>@examples{safe}</div>
            </div>
            """,
        )
    )

    p.xaxis.axis_label = "Win rate"
    p.xaxis.formatter = NumeralTickFormatter(format="0%")
    p.yaxis.axis_label = ""
    p.x_range.start = 0
    p.outline_line_color = None
    return p


corp_stats = _agg_identity_winrate(cutter_results, who="corp")
runner_stats = _agg_identity_winrate(cutter_results, who="runner")

TOP_N = 30
corp_top = _top_n(corp_stats, TOP_N)
runner_top = _top_n(runner_stats, TOP_N)

p_corp = _hbar_chart(corp_top, f"Corp")
p_runner = _hbar_chart(runner_top, f"Runner")

footer = Div(
    text=f"""
    <h3>Notes!</h3>
    <p>Tournaments: {", ".join(tournaments)}</p>
    <p>Split type: {split_type} ({unique_players} players)</p>
    <p>Low game cut off: {low_game_cutoff_nb:.1f} games ({LOW_GAME_CUTOFF_P*100:.1f}% of {len(cutter_results)} total games)</p>
    <p>Byes and IDs are excluded.</p>
    """,
    sizing_mode="stretch_width",
)

layout = column(p_corp, p_runner, footer, sizing_mode="stretch_width")

show(layout)
# save(layout)

Loading BokehJS ...

In [5]:
output_file("cutter_winrates.html", title="Cutter Winrates")
save(layout)

'f:\\personal\\repos\\netrunner_results\\src\\visualisations\\cutter_winrates.html'

In [6]:
# idx = (cutter_results["corp_id"] == "The Zwicky Group: Invisible Hands") & (cutter_results["corp_is_cutter"])
idx = (cutter_results["corp_id"] == "Haas-Bioroid: Precision Design") & (cutter_results["corp_is_cutter"])

cutter_results[idx]

,date,meta,region,location,level,tournament_id,style,tournament,phase,round,table,corp_player,corp_id,corp_faction,result,runner_player,runner_id,runner_faction,corp_is_cutter,runner_is_cutter
50,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,2,1,Pinsel,Haas-Bioroid: Precision Design,HB,runner,Saerleon,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,True,False
59,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,2,10,simen,Haas-Bioroid: Precision Design,HB,corp,giventofly,Sebastião Souza Pessoa: Activist Organizer,Anarch,True,False
102,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,3,1,simen,Haas-Bioroid: Precision Design,HB,corp,Baserton,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,True,False
171,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,4,18,Pinsel,Haas-Bioroid: Precision Design,HB,corp,KingSolomon,"Barry ""Baz"" Wong: Tri-Maf Veteran",Criminal,True,False
209,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,5,4,Pinsel,Haas-Bioroid: Precision Design,HB,corp,"Porkobolo, the roaring pig",Zahya Sadeghi: Versatile Smuggler,Criminal,True,False
260,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,6,3,simen,Haas-Bioroid: Precision Design,HB,corp,MotionBlur,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,True,True
362,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,8,1,simen,Haas-Bioroid: Precision Design,HB,runner,davz131,MuslihaT: Multifarious Marketeer,Criminal,True,True
376,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,8,15,Pinsel,Haas-Bioroid: Precision Design,HB,runner,mallory (l0velace),MuslihaT: Multifarious Marketeer,Criminal,True,True
415,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,9,3,simen,Haas-Bioroid: Precision Design,HB,runner,ZomZraft,Lat: Ethical Freelancer,Shaper,True,True
466,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,10,4,Pinsel,Haas-Bioroid: Precision Design,HB,corp,AceEmpress,Dewi Subrotoputri: Pedagogical Dhalang,Shaper,True,False


In [7]:
standings

,date,meta,region,location,level,tournament_id,style,tournament,top_cut_rank,swiss_rank,...,runner_losses,runner_draws,matchPoints,SoS,xSoS,corp_ID,corp_faction,runner_ID,runner_faction,nrdb_id
0,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,5.0,1,...,2,0,26,1.980556,1.772676,Nebula Talent Management: Making Stars,NBN,Hoshiko Shiro: Untold Protagonist,Anarch,35094.0
1,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,9.0,2,...,3,0,24,1.800000,1.686451,AU Co.: The Gold Standard in Clones,Jinteki,Lat: Ethical Freelancer,Shaper,40402.0
2,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,3.0,3,...,4,0,24,1.580000,1.652329,AU Co.: The Gold Standard in Clones,Jinteki,Zahya Sadeghi: Versatile Smuggler,Criminal,2452.0
3,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,1.0,4,...,1,0,23,1.822222,1.762840,AU Co.: The Gold Standard in Clones,Jinteki,MuslihaT: Multifarious Marketeer,Criminal,37519.0
4,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,10.0,5,...,1,0,23,1.768571,1.681456,AU Co.: The Gold Standard in Clones,Jinteki,MuslihaT: Multifarious Marketeer,Criminal,17901.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,2025-08-30,25.08,NaN,online,continental championship,cobra-4149,SSS,EMEA Online Continentals 2025,NaN,97,...,3,0,3,1.117143,1.310270,MirrorMorph: Endless Iteration,HB,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,NaN
406,2025-08-30,25.08,NaN,online,continental championship,cobra-4149,SSS,EMEA Online Continentals 2025,NaN,98,...,2,0,0,2.000000,1.247500,Nebula Talent Management: Making Stars,NBN,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,NaN
407,2025-08-30,25.08,NaN,online,continental championship,cobra-4149,SSS,EMEA Online Continentals 2025,NaN,99,...,1,0,0,1.700000,1.453869,AU Co.: The Gold Standard in Clones,Jinteki,Magdalene Keino-Chemutai: Cryptarchitect,Shaper,21959.0
408,2025-08-30,25.08,NaN,online,continental championship,cobra-4149,SSS,EMEA Online Continentals 2025,NaN,100,...,4,0,0,1.362245,1.235465,Haas-Bioroid: Precision Design,HB,MuslihaT: Multifarious Marketeer,Criminal,31178.0
